### Home task: pandas 

## Question 1

- Load the energy data from the file [Energy Indicators.xls](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls).
It is a list of indicators of energy supply and renewable electricity production from the United Nations for the year 2013.


- It should be put into a DataFrame with the variable name of "energy"


- Make sure to exclude the footer and header information from the datafile.


- The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:<br>
`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`


- Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule).


- For all countries which have missing data (e.g. data with `...`) make sure this is reflected as `np.NaN` values.


- Rename the following list of countries (for use in later questions):
    - `Republic of Korea`: `South Korea`,
    - `United States of America`: `United States`,
    - `United Kingdom of Great Britain and Northern Ireland`: `United Kingdom`,
    - `China, Hong Kong Special Administrative Region`: `Hong Kong`


- There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, e.g.:
    - `Bolivia (Plurinational State of)` should be `Bolivia`,
    - `Switzerland17` should be `Switzerland`.


- Next, load the GDP data from the file ["world_bank.csv"](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). 
It is a csv containing countries' GDP from 1960 to 2015 from World Bank. Call this DataFrame "GDP"


- Make sure to skip the header, and rename the following list of countries:
    - `Korea, Rep.`: `South Korea`,
    - `Iran, Islamic Rep.`: `Iran`,
    - `Hong Kong SAR, China`: `Hong Kong`


- Finally, load the "Sciamgo Journal and Country Rank data for [Energy Engineering and Power Technology"](http://www.scimagojr.com/countryrank.php?category=2102). It ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame "ScimEn"


- Join the three datasets: Energy, GDP, and ScimEn into a new dataset (using the intersection of country names). Use only the 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15).


- The index of this DataFrame should be the name of the country, and the columns should be<br>
`['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', 2012', '2013', '2014', '2015']`

Function "answer_one" should return the resulted DataFrame (20 columns and 15 entries)

In [1]:
import pandas as pd
import numpy as np

def answer_one():
    file_name = 'Energy Indicators.xls'
    En_df = pd.read_excel(file_name, skiprows=16, skipfooter=283-245)
    En_df = En_df.drop(['Unnamed: 0', 'Unnamed: 1'], axis=1)
    En_df.rename(columns={'Unnamed: 2': 'Country', 'Renewable Electricity Production':'% Renewable'}, inplace=True)
    En_df = En_df.drop(En_df.index[0])

    for col in En_df.columns[1:-1]:
        En_df[col] = En_df[col].mask(En_df[col].str.isnumeric() == False, np.nan).astype(float)

    En_df['Energy Supply'] = En_df['Energy Supply']*1000000

    En_df['Country'] = En_df['Country'].str.replace('\d+', '') # delete numbers
    En_df['Country'] = En_df['Country'].str.replace(r' \([^)]*\)',"") #delete 
    En_df['Country'] = En_df['Country'].replace({'Republic of Korea':'South Korea',
                          'United States of America': 'United States',
                          'United Kingdom of Great Britain and Northern Ireland': 'United Kingdom',
                          'China, Hong Kong Special Administrative Region': 'Hong Kong'})

    GDP = pd.read_csv('API_NY.GDP.MKTP.CD_DS2_en_csv_v2_2531304.csv', skiprows=4)
    GDP['Country Name'] = GDP['Country Name'].replace({'Korea, Rep.': 'South Korea',
                                                    'Iran, Islamic Rep.': 'Iran',
                                                    'Hong Kong SAR, China': 'Hong Kong'})

    GDP.rename(columns={'Country Name': 'Country'}, inplace= True)
    ScimEn = pd.read_excel('scimagojr country rank 1996-2020.xlsx')

    gdp_year_list = np.array(['Country','2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015'])

    for i in GDP.columns:
        if i not in gdp_year_list:
            GDP = GDP.drop(i, axis=1)

    ScimEn = ScimEn.nsmallest(15, 'Rank')
    GDP_ScimEn = pd.merge (GDP, ScimEn, how='inner', on='Country')
    En_GDP_ScimEn = pd.merge(GDP_ScimEn, En_df, how='inner', on='Country')
    En_GDP_ScimEn.set_index('Country', inplace=True)
    En_GDP_ScimEn.drop('Region', axis=1, inplace=True)
    En_GDP_ScimEn.shape
    
    return En_GDP_ScimEn

## Answer the following questions in the context of only the top 15 countries by Scimagojr Rank (aka the DataFrame returned by `answer_one()`)

### Question 2
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [4]:
def answer_two():
    En_GDP_ScimEn = answer_one()
    avg = 0
    k = 0
    for col in En_GDP_ScimEn.columns:
        if col.isnumeric() == True:
            k += 1
            avg += En_GDP_ScimEn[col]
        En_GDP_ScimEn['avgGDP'] = avg/k
        
    top_15 = En_GDP_ScimEn['avgGDP'].sort_values(ascending=False)
    return top_15

answer_two()

<ipython-input-1-78c34abc58f4>:16: FutureWarning: The default value of regex will change from True to False in a future version.
  En_df['Country'] = En_df['Country'].str.replace('\d+', '') # delete numbers
<ipython-input-1-78c34abc58f4>:17: FutureWarning: The default value of regex will change from True to False in a future version.
  En_df['Country'] = En_df['Country'].str.replace(r' \([^)]*\)',"") #delete


Country
United States         1.566966e+13
China                 6.927702e+12
Japan                 5.177131e+12
Germany               3.518225e+12
United Kingdom        2.776607e+12
France                2.687371e+12
Italy                 2.139775e+12
Brazil                1.988885e+12
Russian Federation    1.666746e+12
Canada                1.613835e+12
India                 1.602352e+12
Spain                 1.398587e+12
South Korea           1.221372e+12
Australia             1.206526e+12
Iran                  4.389286e+11
Name: avgGDP, dtype: float64

### Question 3
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [16]:
def answer_three():
    En_GDP_ScimEn = answer_one()
    top_15 = answer_two()
    ind = top_15.index[5]
    result = En_GDP_ScimEn.loc[ind]['2015'] - En_GDP_ScimEn.loc[ind]['2006']
    return result

answer_three()

<ipython-input-1-78c34abc58f4>:16: FutureWarning: The default value of regex will change from True to False in a future version.
  En_df['Country'] = En_df['Country'].str.replace('\d+', '') # delete numbers
<ipython-input-1-78c34abc58f4>:17: FutureWarning: The default value of regex will change from True to False in a future version.
  En_df['Country'] = En_df['Country'].str.replace(r' \([^)]*\)',"") #delete


119614244263.37988

### Question 4

Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [18]:
def answer_four():
    En_GDP_ScimEn = answer_one()
    En_GDP_ScimEn['ratio_of_citations'] = En_GDP_ScimEn['Self-citations']/En_GDP_ScimEn['Citations']
    En_GDP_ScimEn['ratio_of_citations'].max()
    result = En_GDP_ScimEn[En_GDP_ScimEn['ratio_of_citations'] == En_GDP_ScimEn['ratio_of_citations'].max()].index[0], En_GDP_ScimEn['ratio_of_citations'].max()
    return result

answer_four()

<ipython-input-1-78c34abc58f4>:16: FutureWarning: The default value of regex will change from True to False in a future version.
  En_df['Country'] = En_df['Country'].str.replace('\d+', '') # delete numbers
<ipython-input-1-78c34abc58f4>:17: FutureWarning: The default value of regex will change from True to False in a future version.
  En_df['Country'] = En_df['Country'].str.replace(r' \([^)]*\)',"") #delete


('China', 0.6912289816173135)

### Question 5

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [25]:
def answer_five():
    En_GDP_ScimEn = answer_one()
    En_GDP_ScimEn['est_population'] = En_GDP_ScimEn['Energy Supply']/En_GDP_ScimEn['Energy Supply per capita']
    result = En_GDP_ScimEn['est_population'].sort_values(ascending=False).index[2]
    return result, En_GDP_ScimEn

answer_five()[0]

<ipython-input-1-78c34abc58f4>:16: FutureWarning: The default value of regex will change from True to False in a future version.
  En_df['Country'] = En_df['Country'].str.replace('\d+', '') # delete numbers
<ipython-input-1-78c34abc58f4>:17: FutureWarning: The default value of regex will change from True to False in a future version.
  En_df['Country'] = En_df['Country'].str.replace(r' \([^)]*\)',"") #delete


'United States'

### Question 6
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*


In [26]:
def answer_six():
    En_GDP_ScimEn = answer_five()[1]
    En_GDP_ScimEn['citable_doc_per_person'] = En_GDP_ScimEn['Citable documents']/En_GDP_ScimEn['est_population']
    result = En_GDP_ScimEn[['citable_doc_per_person', 'Energy Supply per capita']].corr().iloc[0,1]
    return result, En

answer_six()

<ipython-input-1-78c34abc58f4>:16: FutureWarning: The default value of regex will change from True to False in a future version.
  En_df['Country'] = En_df['Country'].str.replace('\d+', '') # delete numbers
<ipython-input-1-78c34abc58f4>:17: FutureWarning: The default value of regex will change from True to False in a future version.
  En_df['Country'] = En_df['Country'].str.replace(r' \([^)]*\)',"") #delete


0.7434709127726778

### Question 7
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [27]:
def answer_seven():
    En_GDP_ScimEn = answer_five()[1]
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    
    group_by = En_GDP_ScimEn.groupby(by=ContinentDict, axis=0)
    new_df = pd.DataFrame()
    new_df['size'] = group_by.size()
    new_df['sum'] = group_by['est_population'].sum()
    new_df['mean'] = group_by['est_population'].mean()
    new_df['std'] = group_by['est_population'].std()
    
    return new_df

answer_seven()

<ipython-input-1-78c34abc58f4>:16: FutureWarning: The default value of regex will change from True to False in a future version.
  En_df['Country'] = En_df['Country'].str.replace('\d+', '') # delete numbers
<ipython-input-1-78c34abc58f4>:17: FutureWarning: The default value of regex will change from True to False in a future version.
  En_df['Country'] = En_df['Country'].str.replace(r' \([^)]*\)',"") #delete


,size,sum,mean,std
Asia,5,2.898666e+09,5.797333e+08,6.790979e+08
Australia,1,2.331602e+07,2.331602e+07,NaN
Europe,6,4.579297e+08,7.632161e+07,3.464767e+07
North America,2,3.528552e+08,1.764276e+08,1.996696e+08
South America,1,2.059153e+08,2.059153e+08,NaN
